# Exploring the Etsy Crepe Paper Flower Market

In [5]:
import pandas as pd
import numpy as np

In [6]:
listings_main = pd.read_csv('nov_2020/listings_main.csv')
listings_materials = pd.read_csv('nov_2020/listings_materials.csv')
listings_style = pd.read_csv('nov_2020/listings_style.csv')
listings_tags = pd.read_csv('nov_2020/listings_tags.csv')
listings_taxonomy = pd.read_csv('nov_2020/listings_taxonomy.csv')

## Exploration Outline

**Product Type Distribution**
- Proportion of wedding and non-wedding products
- Proportion of pre-made and made-to-order products
- Proportion of combination of above-mentioned attributes


**Price Distribution**
 - Price distribution of all products; median price
 - Price distributions wedding and non-wedding products, median price of both
 - Price distribution of pre-made and made-to-order products, median price of both
 - Price distribution of combination of above-mentioned attributes, median prices of all
 
 
**Listing Trends**
- Top five materials; Distribution of top 10 materials
- Top five styles; Distribution of top 10 styles
- Top five tags; Distribution of top 10 tags
- Top five taxonomies; Distribution of top 10 taxonomies
- Correlation of taxonomies and price; multivariate with wedding attributes
- Correlation of styles and price; multivariate with wedding attributes


**Inference**
- Null: The median price of wedding products is equal to that of non-wedding products
- Null: The median price of made-to-order products is equal to that of pre-made products
- Regression: variables that inform price (wedding, preparation, taxonomy_path, style)


**Insights**
- What is the overall range of prices for these types of products and what are the most popular uses and styles?
- What are the rankings of median price of products by wedding or preparation type? Are these prices actually different and not a function of the sample size?
- What other variables are related to price? (from regression analysis & correlation)

## 1. Product Type Distribution

### Proportion of wedding and non-wedding products

In [7]:
wedding_tax = listings_taxonomy[listings_taxonomy.taxonomy_path == 'weddings']

listings_main[listings_main.listing_id.isin(wedding_tax.listing_id)]

,listing_id,user_id,title,description,price,currency_code,quantity,url,views,num_favorers,who_made,when_made,recipient,occasion,is_vintage
7,559445220,8314653,handmade bridal/bridesmaid italian crepe paper...,bridal crepe paper bouquet that lasts a lifeti...,42.00,usd,10,https://www.etsy.com/listing/559445220/handmad...,130,9,i_did,2010_2019,NaN,NaN,False
38,655648719,6078115,wedding/special event corsage or boutonniere: ...,want something extra special for your special ...,20.00,usd,50,https://www.etsy.com/listing/655648719/wedding...,152,8,i_did,made_to_order,unisex_adults,prom,False
56,525879481,25047394,"cinco de mayo, 12 paper flowers, dia de los mu...","12 small crepe paper flowers, day of the dead,...",7.99,usd,4,https://www.etsy.com/listing/525879481/cinco-d...,3302,503,i_did,made_to_order,women,day_of_the_dead,False
57,751114246,25047394,"cinco de mayo, 24 paper flowers, mexican flowe...","24 small crepe paper flowers, day of the dead,...",15.50,usd,133,https://www.etsy.com/listing/751114246/cinco-d...,306,28,i_did,made_to_order,women,day_of_the_dead,False
58,512069218,25047394,"cinco de mayo, 12 paper flowers, mexican flowe...","12 small crepe paper flowers, day of the dead,...",7.99,usd,53,https://www.etsy.com/listing/512069218/cinco-d...,3053,274,i_did,made_to_order,women,day_of_the_dead,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4063,884043063,212486637,"paper poppies set, paper flower bouquet, paper...",this pink paper poppies made of high quality i...,24.00,usd,1,https://www.etsy.com/listing/884043063/paper-p...,15,3,i_did,made_to_order,NaN,NaN,False
4068,657359277,20777462,bridal paper flower bouquet. first paper anniv...,treat yourself with gorgeous large bouquet wit...,599.99,usd,1,https://www.etsy.com/listing/657359277/bridal-...,314,26,i_did,made_to_order,NaN,NaN,False
4083,577527906,138653862,cream paper david austin rose bridesmaids pape...,perfect for any occasion! a great way to make ...,24.00,usd,2,https://www.etsy.com/listing/577527906/cream-p...,891,84,i_did,made_to_order,NaN,NaN,False
4084,457590280,20777462,"giant paper flower for newborn photography, al...",this is listing for 1 extra giant 30 inches in...,119.99,usd,50,https://www.etsy.com/listing/457590280/giant-p...,585,37,i_did,made_to_order,NaN,NaN,False
